## データの取り込みと確認

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import japanize_matplotlib
import seaborn as sns
import pandas as pd
import numpy as np
sns.set(font = "IPAexGothic", style = "white") #グラフのプロパティを設定
plt.rcParams["figure.figsize"] = [10, 5] #グラフの大きさを設定
pd.set_option('display.max_columns', 150) #表示する列数の上限を150に設定
pd.set_option('display.max_rows', 50) #表示する行数の上限を50に設定

ModuleNotFoundError: No module named 'japanize_matplotlib'

In [ ]:
# データの取り込み
df_master = pd.read_csv("news_text_analysis_data.csv")
df_master

In [ ]:
# データの欠損と型の確認(info)
df_master.info()

# データに欠損はなし

In [ ]:
# categoryの確認
sns.countplot(y="category", data=df_master)

# 9カテゴリーの記事データがおおむね均等に存在する(livedoor-hommeだけやや少ない)

In [ ]:
# テキストデータの文字数の分布
text_length = df_master["text"].apply(lambda x:len(x))
display(text_length.describe())
sns.histplot(text_length)

# ほとんどの記事が500~1500文字であることが分かる

In [ ]:
# 文字数が一番少ない記事の確認
df_master[text_length==34]

## 課題1. 各文章データを単語の出現数のデータに変換しましょう

1. MeCabまたはJanomeを使って文章から名詞だけを抽出したリストに変換する  
2. pandasのexplodeメソッドを使って、作成した名詞のリストを展開(UNNEST)する

In [1]:
# 分かち書き関数のテスト用サンプル(環境によって挙動が変わる可能性があるので、エラーが出る場合は修正が必要になります)
import MeCab
wakati = MeCab.Tagger() # インスタンス作成
words_info = wakati.parse("私は早く走りたい").splitlines()[:-1] # 各単語の情報を取得(最後の要素だけ必要ないので[:-1]で除外)
print(words_info)
word_info = words_info[0] # 最初の単語(今回は"私")の情報を抜き出す
print(word_info)
word = word_info.split("\t")[0] # splitメソッドで必要な情報だけ抜き出す
part_of_speech = word_info.split("\t")[4].split("-")[0]
print(word, part_of_speech) # 単語と品詞がそれぞれ抜き出せていればOK

ModuleNotFoundError: No module named 'MeCab'

In [ ]:
# 文章から単語だけ抜き出す関数を作成
import MeCab
import re

def extract_nouns_from_text(text):
  tagger = MeCab.Tagger()
  words = []
  text = re.sub(r"[0-9]+", "", text) # 数字を削除
  for word_info in tagger.parse(text).splitlines()[:-1]:
    word = word_info.split("\t")[0]
    part_of_speech = word_info.split("\t")[4].split("-")[0]
    if part_of_speech == "名詞":
      words.append(word)
  return words

# 関数の動作確認
print(extract_nouns_from_text("今回の課題は文章の分析です"))

In [ ]:
# 文章から名詞だけを抽出し、UNNESTする
df_noun = df_master.copy()
df_noun["noun"] = df_noun["text"].apply(extract_nouns_from_text)
df_noun = df_noun.explode("noun")
df_noun[["text", "category", "noun"]]

## 課題2. 全記事で出現頻度の高い単語は何か？

課題1で作成したデータフレームを集計して、出現頻度の高い単語を抽出する

In [ ]:
#groupbyメソッドを使って各単語の出現数を集計
df_noun.groupby("noun", as_index=False).count()[["noun", "text"]].sort_values("text", ascending=False).head(30)

#"こと"や"年"、"月"など、集計の意味がない単語が多く含まれているため、このままでは解釈しづらい

,noun,text
12043,こと,24629
30137,人,9729
39723,日,9538
37052,年,8176
39788,日本,7114
40006,映画,6308
40348,月,5959
47149,自分,5556
13526,もの,5465
48606,記事,5225


In [ ]:
# 除外ワードリストを作成し、再度集計
except_word_list = ["こと", "年", "月", "日", "もの", "人", "記事", "ため", "時", "一", "S", "D", "関連", "今回", "話題", "中", "気", "今", "前", "度"]
df_noun = df_noun.query('noun not in @except_word_list')
word_cnt = df_noun.groupby("noun", as_index=False).count()[["noun", "text"]]
word_cnt.columns = ["noun", "cnt"]
word_cnt.sort_values("cnt", ascending=False).head(10)

,noun,cnt
39774,日本,7114
39992,映画,6308
47132,自分,5556
35375,女性,4984
31712,円,4954
14207,アプリ,4785
38160,情報,4200
24498,フォン,4026
44337,発売,3906
36188,対応,3871


In [ ]:
# 出現率の高い単語を集計
all_text_count = df_noun["text"].nunique()
word_freq_rate = df_noun.groupby("noun", as_index=False).nunique("text")[["noun", "text"]]
word_freq_rate["text"] = word_freq_rate["text"] / all_text_count
word_freq_rate.columns = ["noun", "freq_rate"]
word_freq_rate.sort_values("freq_rate", ascending=False).head(10)

,noun,freq_rate
39774,日本,0.393487
38160,情報,0.316554
47132,自分,0.316282
40087,時間,0.253596
40348,月日,0.253460
46112,紹介,0.239077
29572,世界,0.237720
35375,女性,0.232972
12721,ところ,0.232564
44337,発売,0.228765


In [ ]:
# 出現数と出現率を同じデータにまとめる
all_category_result = pd.merge(word_cnt, word_freq_rate, on="noun", how="outer")
display(all_category_result.sort_values("cnt", ascending=False).head(10))
display(all_category_result.sort_values("freq_rate", ascending=False).head(10))

,noun,cnt,freq_rate
39774,日本,7114,0.393487
39992,映画,6308,0.175305
47132,自分,5556,0.316282
35375,女性,4984,0.232972
31712,円,4954,0.218046
14207,アプリ,4785,0.143012
38160,情報,4200,0.316554
24498,フォン,4026,0.146676
44337,発売,3906,0.228765
36188,対応,3871,0.183853


,noun,cnt,freq_rate
39774,日本,7114,0.393487
38160,情報,4200,0.316554
47132,自分,5556,0.316282
40087,時間,3860,0.253596
40348,月日,3223,0.253460
46112,紹介,3155,0.239077
29572,世界,3369,0.237720
35375,女性,4984,0.232972
12721,ところ,2576,0.232564
44337,発売,3906,0.228765


### 【結論】
全記事で出現数、出現頻度が多い単語は上記のようになった。  
「映画」や「女性」、「円」、「アプリ」などエンタメから経済系、IT系などで使われがちな言葉がよく使われている。  
出現頻度をみると、「情報」や「紹介」などの単語が多く、情報系や紹介系の記事が多いことも窺える。  
ただこのままでは正直まだよくわからないので、カテゴリー別に集計したデータを見る必要がありそう。

## 課題3. カテゴリー別で見た時に、出現頻度の高い単語はそれぞれ何か？

カテゴリー別に課題2で行った集計を行い、出現頻度の高い単語を抽出する

In [ ]:
#課題2で行った集計を関数化する
def text_summary_making(target_category):
    # 集計したいカテゴリーのデータに絞る
    target_df_noun = df_noun.query('category == @target_category')
    # 出現数を集計
    word_cnt = target_df_noun.groupby("noun", as_index=False).count()[["noun", "text"]]
    word_cnt.columns = ["noun", "cnt"]
    # 出現率を集計
    all_text_count = target_df_noun["text"].nunique()
    word_freq_rate = target_df_noun.groupby("noun", as_index=False).nunique("text")[["noun", "text"]]
    word_freq_rate["text"] = word_freq_rate["text"] / all_text_count
    word_freq_rate.columns = ["noun", "freq_rate"]
    # 出現数と出現率を結合
    df_result = pd.merge(word_cnt, word_freq_rate, on="noun", how="outer")
    return df_result

In [ ]:
# movie-enter
except_word_list2 = ["映画", "作品", "月日", "本作", "MOVIE", "ENTER"]
df_result = text_summary_making("movie-enter")
df_result = df_result.query('noun not in @except_word_list2')
display(df_result.sort_values("cnt", ascending=False).head(10))
display(df_result.sort_values("freq_rate", ascending=False).head(10))

,noun,cnt,freq_rate
8673,公開,2249,0.786452
12169,日本,1289,0.584386
14138,監督,1221,0.493685
7846,世界,1199,0.601607
12244,映像,926,0.468427
16001,賞,823,0.247991
14077,登場,731,0.453502
11203,役,703,0.347876
8875,初,580,0.397245
4000,シーン,576,0.326062


,noun,cnt,freq_rate
8673,公開,2249,0.786452
7846,世界,1199,0.601607
12169,日本,1289,0.584386
14138,監督,1221,0.493685
8620,全国,528,0.492537
12244,映像,926,0.468427
14077,登場,731,0.453502
3716,サイト,468,0.419059
8875,初,580,0.397245
8661,公式,431,0.382319


日本で公開された映画や賞を取った映画の記事が多いことが分かる。  
"初"というワードもよく使われおり、初公開などの組み合わせで使われていることが窺える。

In [ ]:
# it-life-hack
except_word_list2 = ["通", "元", "jp"]
df_result = text_summary_making("it-life-hack")
df_result = df_result.query('noun not in @except_word_list2')
display(df_result.sort_values("cnt", ascending=False).head(10))
display(df_result.sort_values("freq_rate", ascending=False).head(10))

,noun,cnt,freq_rate
7689,写真,1246,0.233065
10408,機能,1235,0.416762
3808,アプリ,1210,0.272101
2780,iPhone,1143,0.354765
1566,PC,1094,0.409874
11018,画面,960,0.274397
8919,対応,921,0.373134
11989,製品,892,0.343284
4698,サービス,840,0.320321
11950,表示,765,0.245695


,noun,cnt,freq_rate
9406,情報,731,0.427095
12258,販売,671,0.421355
10408,機能,1235,0.416762
1566,PC,1094,0.409874
11521,紹介,552,0.390356
4396,クチコミ,340,0.390356
8919,対応,921,0.373134
7818,利用,711,0.370838
2780,iPhone,1143,0.354765
6488,ユーザー,752,0.344432


パソコンやスマホなどのガジェット系の記事が多そう。  
製品の機能や口コミ、販売情報がよく扱われている様子。

In [ ]:
# kaden-channel
except_word_list2 = []
df_result = text_summary_making("kaden-channel")
df_result = df_result.query('noun not in @except_word_list2')
display(df_result.sort_values("cnt", ascending=False).head(10))
display(df_result.sort_values("freq_rate", ascending=False).head(10))

,noun,cnt,freq_rate
4710,ビデオ,942,0.330636
9905,発売,877,0.482081
4124,チェック,844,0.600000
7386,売れ筋,723,0.564162
4218,テレビ,655,0.331792
1254,SALON,570,0.279769
3107,カメラ,567,0.245087
6567,円,560,0.324855
8770,日本,548,0.349133
1894,iPhone,537,0.297110


,noun,cnt,freq_rate
4124,チェック,844,0.600000
7386,売れ筋,723,0.564162
9905,発売,877,0.482081
9933,登場,502,0.411561
8770,日本,548,0.349133
4218,テレビ,655,0.331792
4710,ビデオ,942,0.330636
6567,円,560,0.324855
6086,人気,406,0.310983
9924,発表,467,0.309827


家電の中でもビデオやテレビがよくピックアップされているのが分かる。  
売れ筋商品や発売した商品の情報紹介が多いことが窺える。  
値段も紹介していることから、購買系の情報が多いことも分かる。

In [ ]:
# topic-news
except_word_list2 = ["同", "一方"]
df_result = text_summary_making("topic-news")
df_result = df_result.query('noun not in @except_word_list2')
display(df_result.sort_values("cnt", ascending=False).head(10))
display(df_result.sort_values("freq_rate", ascending=False).head(10))

,noun,cnt,freq_rate
3176,ネット,1605,0.833766
11837,韓国,1071,0.174026
8067,日本,1068,0.420779
7788,掲示,1056,0.720779
6214,声,843,0.628571
7437,情報,729,0.777922
7641,批判,579,0.451948
9468,番組,525,0.259740
3954,ユーザー,524,0.419481
7865,放送,512,0.259740


,noun,cnt,freq_rate
3176,ネット,1605,0.833766
7437,情報,729,0.777922
7788,掲示,1056,0.720779
6214,声,843,0.628571
7641,批判,579,0.451948
8067,日本,1068,0.420779
3954,ユーザー,524,0.419481
7467,意見,343,0.342857
3153,ニュース,438,0.319481
2424,コメント,307,0.283117


ネット上の情報をまとめたトピック記事が多そう。  
韓国やテレビ番組などに対する批判や意見、コメントが紹介されている様子。  

In [ ]:
# livedoor-homme
except_word_list2 = ["万", "以上", "livedoor", "元"]
df_result = text_summary_making("livedoor-homme")
df_result = df_result.query('noun not in @except_word_list2')
display(df_result.sort_values("cnt", ascending=False).head(10))
display(df_result.sort_values("freq_rate", ascending=False).head(10))

,noun,cnt,freq_rate
13332,転職,889,0.298828
3502,ゴルフ,798,0.111328
7440,円,741,0.380859
6885,仕事,706,0.373047
9625,情報,702,0.435547
12499,自分,699,0.451172
6991,会社,633,0.333984
9246,年収,552,0.236328
10223,日本,529,0.400391
10959,求人,448,0.292969


,noun,cnt,freq_rate
12499,自分,699,0.451172
9625,情報,702,0.435547
10223,日本,529,0.400391
10541,東京,351,0.400391
9978,提供,263,0.390625
7440,円,741,0.380859
6885,仕事,706,0.373047
6991,会社,633,0.333984
11394,特集,283,0.330078
10307,時間,357,0.324219


ビジネスマンが気になる単語が多く、ビジネスマンを対象とした記事が多そう。  
主に仕事関連とゴルフ関連がメインであることが窺える。

In [ ]:
# peachy
except_word_list2 = ["方", "月日"]
df_result = text_summary_making("peachy")
df_result = df_result.query('noun not in @except_word_list2')
display(df_result.sort_values("cnt", ascending=False).head(10))
display(df_result.sort_values("freq_rate", ascending=False).head(10))

,noun,cnt,freq_rate
11884,女性,1184,0.489286
10403,円,1004,0.329762
11879,女子,954,0.469048
12878,応募,801,0.188095
16718,自分,796,0.378571
16596,肌,788,0.178571
7725,プレゼント,765,0.283333
12739,当選,634,0.126190
13608,日本,599,0.341667
9764,人気,597,0.416667


,noun,cnt,freq_rate
11884,女性,1184,0.489286
11879,女子,954,0.469048
9764,人気,597,0.416667
5014,サイト,493,0.388095
16718,自分,796,0.378571
16272,紹介,454,0.358333
13608,日本,599,0.341667
10403,円,1004,0.329762
13721,時間,412,0.294048
7725,プレゼント,765,0.283333


女性が好む単語が多く、女性向けの記事であることが分かる。  
女性の肌や人気アイテムなどを紹介したり、応募やプレゼント系のイベントを告知したりする内容が多そう。(女性は応募やプレゼント企画が好き??)

In [ ]:
# sports-watch
except_word_list2 = ["Sports", "Watch"]
df_result = text_summary_making("sports-watch")
df_result = df_result.query('noun not in @except_word_list2')
display(df_result.sort_values("cnt", ascending=False).head(10))
display(df_result.sort_values("freq_rate", ascending=False).head(10))

,noun,cnt,freq_rate
7177,日本,1559,0.673696
10236,選手,1093,0.553829
4121,代表,797,0.367370
9742,試合,765,0.382908
8568,監督,670,0.243063
1595,サッカー,556,0.304107
10324,野球,504,0.228635
7011,放送,495,0.442841
2227,テレビ,462,0.411765
9286,自分,446,0.269700


,noun,cnt,freq_rate
7177,日本,1559,0.673696
10236,選手,1093,0.553829
7011,放送,495,0.442841
2227,テレビ,462,0.411765
9742,試合,765,0.382908
4121,代表,797,0.367370
8476,番組,432,0.312986
1595,サッカー,556,0.304107
2190,チーム,438,0.283019
9286,自分,446,0.269700


スポーツの中ではサッカーの記事が一番多く、二番目は野球。  
試合や選手、テレビ放送の情報に関する記事が多い。   

In [ ]:
# dokujo-tsushin
except_word_list2 = ["ところ", "とき", "方", "話", "そう"]
df_result = text_summary_making("dokujo-tsushin")
df_result = df_result.query('noun not in @except_word_list2')
display(df_result.sort_values("cnt", ascending=False).head(10))
display(df_result.sort_values("freq_rate", ascending=False).head(10))

,noun,cnt,freq_rate
11793,歳,2389,0.676640
8994,女性,2374,0.713464
13801,結婚,2080,0.423475
14264,自分,2046,0.713464
12870,男性,1434,0.484465
12626,独女,1192,0.546605
8978,女,1134,0.372842
6724,仕事,1018,0.459148
12855,男,882,0.298044
13160,相手,862,0.362486


,noun,cnt,freq_rate
14264,自分,2046,0.713464
8994,女性,2374,0.713464
11793,歳,2389,0.676640
12626,独女,1192,0.546605
12870,男性,1434,0.484465
6724,仕事,1018,0.459148
13801,結婚,2080,0.423475
8978,女,1134,0.372842
13160,相手,862,0.362486
8989,女子,786,0.359033


特定の年齢層の女性をターゲットにした記事が多そう。  
全て結婚に関わる単語であり、結婚関連の記事がほとんどであることが推察できる。

In [ ]:
# smax
except_word_list2 = ["MAX", "対応", "更新", "マックス", "エス", "on", "名", "smaxjp"]
df_result = text_summary_making("smax")
df_result = df_result.query('noun not in @except_word_list2')
display(df_result.sort_values("cnt", ascending=False).head(10))
display(df_result.sort_values("freq_rate", ascending=False).head(10))

,noun,cnt,freq_rate
5833,フォン,2739,0.712974
3940,アプリ,2570,0.513203
120,Android,2469,0.724455
5060,ソフトウェア,2022,0.174512
5409,ドコモ,2018,0.435132
10176,画面,1668,0.518944
9095,搭載,1657,0.489093
6294,モデル,1649,0.440873
9664,機能,1578,0.515499
10205,発売,1550,0.454650


,noun,cnt,freq_rate
6457,リンク,916,0.994259
2215,Twitter,1021,0.990815
8055,執筆,868,0.989667
120,Android,2469,0.724455
5833,フォン,2739,0.712974
11646,開発,810,0.567164
11639,開始,1152,0.540758
10176,画面,1668,0.518944
7478,利用,1528,0.518944
9664,機能,1578,0.515499


スマホを中心としたガジェット系記事であることが分かる。  
開発、機能、開始といったワードが多いところをみると、ハードよりもソフトウェアよりな記事が多そう。

### 【結論】  
#### movie-enter  
「公開、日本、登場、賞」などの日本公開や受賞映画に関するワード  
#### it-life-hack  
「PC、iPhone、アプリ、写真、機能」などのガジェットに関するワード  
「製品、機能、口コミ、販売」などの製品の機能や口コミ、販売情報系のワード  
#### kaden-channel  
「ビデオ、テレビ」などの家電ワード  
「売れ筋、発売、チェック、円」などの商品情報紹介ワード  
#### topic-news
「ネット、韓国、番組、放送、批判、意見、コメント」などネット情報系のワード  
#### livedoor-homme
「転職、仕事、年収、会社、ゴルフ」などビジネスマンが気になる系のワード  
#### peachy
「女性、肌、人気、応募、プレゼント」など女性が気になるワード  
#### sports-watch
「試合、選手、テレビ、放送」などスポーツの試合、選手情報系のワード  
サッカーが一番多く、野球が二番目  
#### dokujo-tsushin
「歳、女性、結婚」など特定の年齢層の婚活女性が気になる系のワード  
#### smax
「Android、開発、アプリ、搭載」などITソフトウェア系のワード  


## 分析の結論

#### 今回の分析で、記事のカテゴリーごとに使われるワードに大きな差があることや、現在多く使われている単語が分かった。  

##### movie-enter  
「公開、日本、登場、賞」などの日本公開や受賞映画に関するワード  
##### it-life-hack  
「PC、iPhone、アプリ、写真、機能」などのガジェットに関するワード  
「製品、機能、口コミ、販売」などの製品の機能や口コミ、販売情報系のワード  
##### kaden-channel  
「ビデオ、テレビ」などの家電ワード  
「売れ筋、発売、チェック、円」などの商品情報紹介ワード  
##### topic-news
「ネット、韓国、番組、放送、批判、意見、コメント」などネット情報系のワード  
##### livedoor-homme
「転職、仕事、年収、会社、ゴルフ」などビジネスマンが気になる系のワード  
##### peachy
「女性、肌、人気、応募、プレゼント」など女性が気になるワード  
##### sports-watch
「試合、選手、テレビ、放送」などスポーツの試合、選手情報系のワード  
サッカーが一番多く、野球が二番目  
##### dokujo-tsushin
「歳、女性、結婚」など特定の年齢層の婚活女性が気になる系のワード  
##### smax
「Android、開発、アプリ、搭載」などITソフトウェア系のワード  

#### 今後の展望

この結果をベースに、今後以下のような解析をしていくとより有意義なトレンド分析が可能になる。  
- 前週との差分などをチェック(特に前週より出現数が大幅に伸びた単語を自動で抽出できるようにするとgood)
- 記事の閲覧数やコメント数、掲載順などの情報を追加で取得し、人気記事によく使われているワードを分析する
- 名詞だけでなく動詞なども分析対象に加える
- 特定の単語(たとえばスポーツ記事ならスポーツ名のみ)などに限定して分析することで、トレンドスポーツなどの抽出をする
- 機械学習を活用して今後伸びるワードを予測する